In [52]:
import pickle
import numpy as np
import tensorflow as tf
import keras

import os
import datetime

from keras.layers import Dense, MaxPooling2D, Flatten, Conv2D
from Chess.view import view_board_colour
from Chess.state import Board, construct_board
from Chess.constants import WHITE, BLACK
from sklearn.model_selection import train_test_split

from ipywidgets import interact, fixed
import ipywidgets as widgets

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

In [53]:
np.random.seed(20)

In [54]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [55]:
# From https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
def new_tensorboard_run():
    logdir = os.path.join("tensorboard_logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    return tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

### Visualisation functions

In [56]:
# Functions to manipulate the output
def predict_val(model, val):
    prediction = model.predict(val)
    return np.where(prediction > 0.5, 1, 0)

def predict_raw(model, val):
    return model.predict(val)  

def find_diff(y, pred):
    Y_instance = y
    return np.where(pred == Y_instance, 1, 0)

In [57]:
# Functions to draw the labels and predictions.

sns.set_theme(style="white")
sns.color_palette("crest", as_cmap=True)
sns.set_palette("crest")

def plot_sns(val):
    # From https://seaborn.pydata.org/examples/many_pairwise_correlations.html
    sns.heatmap(val.reshape(8,8))
    plt.show()

def plot(val):
    cmap = colors.ListedColormap(['white', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val.reshape(8, 8), cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));
    plt.show()

def plot_diff(val):
    cmap = colors.ListedColormap(['red', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val.reshape(8,8) , cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));
    plt.show()

In [58]:
def generate_plots(
            n, 
            model = None,
            x = None,
            y = None,
            target=True, 
            prediction=True, 
            raw_prediction=True, 
            diff=True
        ):
    if target:         
        plot(y[n].reshape(1, 64))    
    if prediction:     
        plot(predict_val(model, x[n:n+1]))
    if raw_prediction: 
        plot_sns(predict_raw(model, x[n:n+1]))
    if diff:          
        pred = predict_val(model, x[n:n+1])
        plot_diff(find_diff(y[n:n+1], pred))

In [59]:
class TFModel():
    def __init__(self, name, n_epochs=200, batch_size=50, loss='binary_crossentropy', optimizer='adam'):
        self.name = name
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.loss_func = loss
        self.optimizer = optimizer
        logdir = os.path.join("tensorboard_logs", self.name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        self.tb_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    def compile(self):
        self.model.compile(
            loss=self.loss_func,
            optimizer=self.optimizer,
            metrics=["accuracy", "binary_accuracy"]
        )
    
    @property
    def summary(self):
        return self.model.summary
        
    def fit(self, X, Y):
        return self.model.fit(
            X, 
            Y, 
            epochs=self.n_epochs, 
            batch_size=self.batch_size, 
            callbacks=[self.tb_callback]
        )
    
    def evaluate(self, X, Y):
        self.model.evaluate(
            X, 
            Y, 
            callbacks=[self.tb_callback]
        )
    
    def predict(self, X):
        return self.model.predict(X)

# Basic sequential model

We will begin training a basic fully-connected network specified with Keras. This is primarily meant as a test and to define some useful functions (such as those for visualising our outputs). We will start by preprocessing, then defining a model, training and evaluating.

### Data generation
The script `generate_data.py` will open a list of PGN (portable game notation) chess games and run through them, getting from it some state during the game. This state contains information on the location, type and colour of the pieces as well as the legal moves in the position. However, it does not have any information on the strength of those moves, meaning we can only predict the *legal* moves and not rank them.

The games which are generated are saved in a `txt` file as FEN (Forsyth-Edwards Notation) strings, which encapsulate all the information needed to recreate a position in chess. We can use these strings to reconstruct the boards and gain access to the information we need. 

The current file being used was obtained from Lichess and is a set of ~121,000 games played online over the course of 2013. They are a mixture of classical, blitz and bullet games played at a variety of different levels.

In [60]:
with open("generated_data/lichess_db_standard_rated_2013-01.txt", 'r') as f:
    all_boards = f.readlines()
    
# Take a random sample from boards to reduce the training time
idx = np.random.randint(0, len(all_boards), 5000)
all_boards = np.array(all_boards)
boards = all_boards[idx]

board_list = [construct_board(i) for i in boards]

In [61]:
# An example of what one of these positions looks like
view_board_colour(board_list[0])

Black to move - turn 9
 R     B  Q  K  B     Q 
 P  P  P        P     P 
                N  P    
          P        B    
       P     P          
                        
 P  P           P  P  P 
 R  N        K  B     R 



## Data processing
Now the data needs to be formed into a reasonable type for training. This section will get all the positions which can be moved to as a list for each game, and all the positions of the pieces for each board as training data. Initally, this won't account for the piece type, which side is to move, or the different types of pieces and moves.

In [62]:
# Uses set notation to get only unique moves
valid_move_list = [list({i for i in board.moves.all_valid}) for board in board_list] 

# Get all the piece types and positions from the location map of each board (training data)
piece_position_list = [{position: piece.kind for position, piece in board.loc_map.items()} for board in board_list]

In [63]:
# Encode the location of any piece as a 1, and all other positions as 0
feature = []
for instance in piece_position_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    feature.append(array.flatten())

In [64]:
# We encode the valid "destination" squares with a 1 and invalid with a 0
target = []
for instance in valid_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    target.append(array.flatten())

In [65]:
X = np.array(feature)
Y = np.array(target)

# Split the data using sklearn train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [66]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(4000, 64)
(4000, 64)
(1000, 64)
(1000, 64)


Beginning with a simple sequential model, which takes in a 1x64 array of features and outputs a prediction at each of the 64 squares.

In [67]:
fully_connected_model = TFModel("fully_connected")

fc_model_inputs = keras.Input(shape=(64, ))
fc_model = Dense(128, activation='relu')(fc_model_inputs)
fc_model = Dense(256, activation='relu')(fc_model)
fc_model_outputs = Dense(64, activation='sigmoid')(fc_model)

fully_connected_model.model = keras.Model(inputs=fc_model_inputs, outputs=fc_model_outputs)
fully_connected_model.compile()
fully_connected_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_16 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_17 (Dense)             (None, 64)                16448     
Total params: 57,792
Trainable params: 57,792
Non-trainable params: 0
_________________________________________________________________


2021-12-13 21:48:07.572523: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 21:48:07.572537: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 21:48:07.573629: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [68]:
fully_connected_model.fit(X_train, Y_train)

Epoch 1/200
 1/80 [..............................] - ETA: 46s - loss: 0.7001 - accuracy: 0.0400 - binary_accuracy: 0.4950

2021-12-13 21:48:08.097078: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-13 21:48:08.248156: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 21:48:08.248167: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 21:48:08.282877: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-13 21:48:08.287493: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-13 21:48:08.292358: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tensorboard_logs/fully_connected_20211213-214807/train/plugins/profile/2021_12_13_21_48_08
2021-12-13 21:48:08.293777: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tensorboard_logs/fully_connected_20211213-214807/train/plugi

19/80 [======>.......................] - ETA: 0s - loss: 0.6635 - accuracy: 0.0096 - binary_accuracy: 0.6015

2021-12-13 21:48:08.300449: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tensorboard_logs/fully_connected_20211213-214807/train/plugins/profile/2021_12_13_21_48_08
2021-12-13 21:48:08.300680: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for memory_profile.json.gz to tensorboard_logs/fully_connected_20211213-214807/train/plugins/profile/2021_12_13_21_48_08/Elliss-MacBook-Air.local.memory_profile.json.gz
2021-12-13 21:48:08.301417: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: tensorboard_logs/fully_connected_20211213-214807/train/plugins/profile/2021_12_13_21_48_08Dumped tool data for xplane.pb to tensorboard_logs/fully_connected_20211213-214807/train/plugins/profile/2021_12_13_21_48_08/Elliss-MacBook-Air.local.xplane.pb
Dumped tool data for overview_page.pb to tensorboard_logs/fully_connected_20211213-214807/train/plugins/profile/2021_12_13_21_48_08/Elliss-MacBook-Air.local.over

80/80 [==============================] - 1s 9ms/step - loss: 0.6224 - accuracy: 0.0058 - binary_accuracy: 0.6662
Epoch 2/200
80/80 [==============================] - 1s 7ms/step - loss: 0.5585 - accuracy: 0.0098 - binary_accuracy: 0.7212
Epoch 3/200
80/80 [==============================] - 1s 7ms/step - loss: 0.5335 - accuracy: 0.0105 - binary_accuracy: 0.7302
Epoch 4/200
80/80 [==============================] - 1s 8ms/step - loss: 0.5126 - accuracy: 0.0116 - binary_accuracy: 0.7398
Epoch 5/200
80/80 [==============================] - 1s 7ms/step - loss: 0.5013 - accuracy: 0.0165 - binary_accuracy: 0.7458
Epoch 6/200
80/80 [==============================] - 1s 7ms/step - loss: 0.4872 - accuracy: 0.0125 - binary_accuracy: 0.7533
Epoch 7/200
80/80 [==============================] - 1s 7ms/step - loss: 0.4841 - accuracy: 0.0191 - binary_accuracy: 0.7551
Epoch 8/200
80/80 [==============================] - 1s 7ms/step - loss: 0.4758 - accuracy: 0.0154 - binary_accuracy: 0.7603
Epoch 9/200


80/80 [==============================] - 1s 10ms/step - loss: 0.3208 - accuracy: 0.0550 - binary_accuracy: 0.8644
Epoch 67/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3184 - accuracy: 0.0517 - binary_accuracy: 0.8652
Epoch 68/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3174 - accuracy: 0.0591 - binary_accuracy: 0.8671
Epoch 69/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3141 - accuracy: 0.0624 - binary_accuracy: 0.8669
Epoch 70/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3138 - accuracy: 0.0514 - binary_accuracy: 0.8684
Epoch 71/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3105 - accuracy: 0.0532 - binary_accuracy: 0.8691
Epoch 72/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3105 - accuracy: 0.0608 - binary_accuracy: 0.8703
Epoch 73/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3110 - accuracy: 0.0526 - binary_accuracy: 0.8695
Epoc

80/80 [==============================] - 1s 8ms/step - loss: 0.2462 - accuracy: 0.0780 - binary_accuracy: 0.9011
Epoch 131/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2420 - accuracy: 0.0718 - binary_accuracy: 0.9031
Epoch 132/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2431 - accuracy: 0.0665 - binary_accuracy: 0.9027
Epoch 133/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2437 - accuracy: 0.0735 - binary_accuracy: 0.9019
Epoch 134/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2425 - accuracy: 0.0810 - binary_accuracy: 0.9031
Epoch 135/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2446 - accuracy: 0.0677 - binary_accuracy: 0.9007
Epoch 136/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2416 - accuracy: 0.0677 - binary_accuracy: 0.9038
Epoch 137/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2392 - accuracy: 0.0781 - binary_accuracy: 0.904

80/80 [==============================] - 1s 7ms/step - loss: 0.2010 - accuracy: 0.0969 - binary_accuracy: 0.9221
Epoch 195/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2031 - accuracy: 0.0902 - binary_accuracy: 0.9204
Epoch 196/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2058 - accuracy: 0.0864 - binary_accuracy: 0.9199
Epoch 197/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2029 - accuracy: 0.0802 - binary_accuracy: 0.9205
Epoch 198/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2055 - accuracy: 0.0856 - binary_accuracy: 0.9187
Epoch 199/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2026 - accuracy: 0.0853 - binary_accuracy: 0.9204
Epoch 200/200
80/80 [==============================] - 1s 9ms/step - loss: 0.1999 - accuracy: 0.0933 - binary_accuracy: 0.9219


In [69]:
fully_connected_model.evaluate(X_test, Y_test)

14/32 [============>.................] - ETA: 0s - loss: 0.9633 - accuracy: 0.0424 - binary_accuracy: 0.7139

2021-12-13 21:50:19.765426: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 1s 9ms/step - loss: 0.9749 - accuracy: 0.0430 - binary_accuracy: 0.7156


In [70]:
%tensorboard --logdir tensorboard_logs

Reusing TensorBoard on port 6006 (pid 61435), started 0:59:15 ago. (Use '!kill 61435' to kill it.)

## Visualising the output
Now both the raw output of the model and the binarised version can be displayed interactively.

In [71]:
# From https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
interact(
    generate_plots,
    model = fixed(fully_connected_model),
    x = fixed(X_test),
    y = fixed(Y_test),
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)

interactive(children=(IntSlider(value=0, description='n', max=199, step=5), Checkbox(value=False, description=…

<function __main__.generate_plots(n, model=None, x=None, y=None, target=True, prediction=True, raw_prediction=True, diff=True)>

This widget lets us look through the validation set predictions and compare what the model is outputting compared to what the equivilant label is. What we can begin to see from here is that the model does have some predictive power on pieces which do not move great distances up and down the board. However for pieces that can travel the entire diagonal, vertical or horizontal distance of the board (such as rooks, bishops and queens) there is a high margin of error and these motions are rarely predicted at all, indicating they are tough to learn.

This may also be indicitve of some bias in the training data - almost every position has pawns which can be moved in it whereas the major pieces are more often closed in and obscured behind other pieces. This imbalance may contribute to not learning how the pieces move properly.

## Hyperparameters

This model has many hyperparemters already, including:
- The number of epochs
- The size of each batch
- The optimizer used
- The number of parameters in each hidden layer
- The number of hidden layers

It is also more than reasonable to think of adding other optimization techniques such as regularization.
To help optimize these parameters we can use a grid search algorithm. 

# Convolutional design

We can also approach this problem with a convolutional network. These are designed for image processing and operate by sliding a kernel over an image, performing a "convolution" of the kernel on the data. It is particuarly good at identifying patterns, so let us test this out!

In [84]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [85]:
def create_fc_model():
    # Create a model
    fc_model_inputs = keras.Input(shape=(64, ))
    fc_model = Dense(128, activation='relu')(fc_model_inputs)
    fc_model = Dense(256, activation='relu')(fc_model)
    fc_model_outputs = Dense(64, activation='sigmoid')(fc_model)

    model = keras.Model(inputs=fc_model_inputs, outputs=fc_model_outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'binary_accuracy'])
    return model

In [ ]:
def grid_search(build_fn, param_grid):
    model = KerasClassifier(build_fn=build_fn, verbose=1)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
    grid_Result = grid.fit(X, Y)
    
    print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
    means = grid_result.cv_results_['mean_test_score']
    sdev = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for m, s, p in zip(means, sdev, params):
        print(f"{m} ({s}) with: {p}")

In [ ]:
# First we search with a coarse grid and find the best parameters for the number of epochs and batch size
# This definitely should be expanded to include the optimizer, learning rate and regularization
param_grid = {
    "epochs": [10, 100, 500],
    "batch_size": [5, 50]
}
model = KerasClassifier(build_fn=create_fc_model, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
grid_result = grid.fit(X, Y)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Init Plugin
Init Plugin
Init Plugin
Init Plugin
Init Plugin
Init Plugin
Init Plugin
Init Graph Optimizer
Init Graph Optimizer
Init Graph Optimizer
Init Graph Optimizer
Init Graph Optimizer
Init Graph Optimizer
Init Graph Optimizer
Init Kernel
Init Kernel
Init Kernel
Init Kernel
Init Kernel
Init Kernel
Init Kernel
Init Plugin
Init Graph Optimizer
Init Kernel
Metal device set to: Apple M1
Metal device set to: Apple M1
Metal device set to: Apple M1
Metal device set to: Apple M1
Metal device set to: Apple M1
Metal device set to: Apple M1
Metal device set to: Apple M1
Metal device set to: Apple M1


2021-12-13 23:15:39.602512: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-13 23:15:39.603005: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-13 23:15:39.603241: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-12-13 23:15:39.603436: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-13 23:15:39

Epoch 1/10
  9/667 [..............................] - ETA: 15s - loss: 0.6894 - accuracy: 0.1423 - binary_accuracy: 0.5297

2021-12-13 23:15:59.632858: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


  6/667 [..............................] - ETA: 17s - loss: 0.7072 - accuracy: 0.0817 - binary_accuracy: 0.4748

...

667/667 [==============================] - 11s 16ms/step - loss: 0.0910 - accuracy: 0.0964 - binary_accuracy: 0.9653
Epoch 121/500
667/667 [==============================] - 10s 15ms/step - loss: 0.0773 - accuracy: 0.1138 - binary_accuracy: 0.9708
Epoch 133/500
667/667 [==============================] - 10s 14ms/step - loss: 0.0743 - accuracy: 0.1163 - binary_accuracy: 0.9721
Epoch 134/500
667/667 [==============================] - 9s 14ms/step - loss: 0.0889 - accuracy: 0.1165 - binary_accuracy: 0.9663
Epoch 123/500
667/667 [==============================] - 9s 14ms/step - loss: 0.0738 - accuracy: 0.1251 - binary_accuracy: 0.9724
Epoch 135/500
667/667 [==============================] - 9s 13ms/step - loss: 0.0883 - accuracy: 0.1032 - binary_accuracy: 0.9663
Epoch 124/500
667/667 [==============================] - 9s 14ms/step - loss: 0.0749 - accuracy: 0.1139 - binary_accuracy: 0.9717
Epoch 136/500
667/667 [==============================] - 10s 14ms/step - loss: 0.0867 - accuracy: 0.1

667/667 [==============================] - 9s 14ms/step - loss: 0.0744 - accuracy: 0.1290 - binary_accuracy: 0.9718
Epoch 145/500
667/667 [==============================] - 9s 14ms/step - loss: 0.0606 - accuracy: 0.1484 - binary_accuracy: 0.9775
Epoch 157/500
667/667 [==============================] - 9s 14ms/step - loss: 0.0616 - accuracy: 0.1402 - binary_accuracy: 0.9769
Epoch 158/500
667/667 [==============================] - 8s 13ms/step - loss: 0.0718 - accuracy: 0.1311 - binary_accuracy: 0.9730
Epoch 147/500
667/667 [==============================] - 9s 13ms/step - loss: 0.0590 - accuracy: 0.1525 - binary_accuracy: 0.9782
Epoch 159/500
667/667 [==============================] - 9s 13ms/step - loss: 0.0708 - accuracy: 0.1284 - binary_accuracy: 0.9731
Epoch 149/500
667/667 [==============================] - 9s 13ms/step - loss: 0.0602 - accuracy: 0.1479 - binary_accuracy: 0.9775
Epoch 161/500
667/667 [==============================] - 9s 13ms/step - loss: 0.0597 - accuracy: 0.1540 

667/667 [==============================] - 9s 13ms/step - loss: 0.0508 - accuracy: 0.1765 - binary_accuracy: 0.9809
Epoch 182/500
667/667 [==============================] - 9s 14ms/step - loss: 0.0586 - accuracy: 0.1569 - binary_accuracy: 0.9785
Epoch 171/500
667/667 [==============================] - 9s 14ms/step - loss: 0.0513 - accuracy: 0.1638 - binary_accuracy: 0.9809
Epoch 183/500
667/667 [==============================] - 9s 13ms/step - loss: 0.0595 - accuracy: 0.1605 - binary_accuracy: 0.9778
Epoch 172/500
667/667 [==============================] - 9s 14ms/step - loss: 0.0494 - accuracy: 0.1864 - binary_accuracy: 0.9817
Epoch 184/500
667/667 [==============================] - 9s 13ms/step - loss: 0.0600 - accuracy: 0.1675 - binary_accuracy: 0.9774
Epoch 173/500
667/667 [==============================] - 9s 13ms/step - loss: 0.0477 - accuracy: 0.1787 - binary_accuracy: 0.9823
Epoch 185/500
667/667 [==============================] - 10s 15ms/step - loss: 0.0496 - accuracy: 0.1785

667/667 [==============================] - 9s 14ms/step - loss: 0.0495 - accuracy: 0.1863 - binary_accuracy: 0.9816
Epoch 194/500
667/667 [==============================] - 9s 13ms/step - loss: 0.0414 - accuracy: 0.1933 - binary_accuracy: 0.9847
Epoch 207/500
667/667 [==============================] - 9s 13ms/step - loss: 0.0478 - accuracy: 0.1837 - binary_accuracy: 0.9824
Epoch 195/500
667/667 [==============================] - 8s 12ms/step - loss: 0.0421 - accuracy: 0.2032 - binary_accuracy: 0.9847
Epoch 208/500
667/667 [==============================] - 8s 12ms/step - loss: 0.0463 - accuracy: 0.1917 - binary_accuracy: 0.9831
Epoch 196/500
667/667 [==============================] - 8s 12ms/step - loss: 0.0448 - accuracy: 0.1803 - binary_accuracy: 0.9839
Epoch 197/500
667/667 [==============================] - 9s 14ms/step - loss: 0.0490 - accuracy: 0.1942 - binary_accuracy: 0.9820
Epoch 199/500
667/667 [==============================] - 9s 14ms/step - loss: 0.0443 - accuracy: 0.2184 

667/667 [==============================] - 8s 12ms/step - loss: 0.0356 - accuracy: 0.2408 - binary_accuracy: 0.9871
Epoch 232/500
667/667 [==============================] - 8s 12ms/step - loss: 0.0414 - accuracy: 0.2025 - binary_accuracy: 0.9848
Epoch 220/500
667/667 [==============================] - 9s 13ms/step - loss: 0.0362 - accuracy: 0.2354 - binary_accuracy: 0.9870
Epoch 234/500
323/667 [=============>................] - ETA: 4s - loss: 0.0332 - accuracy: 0.2692 - binary_accuracy: 0.9882

In [83]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.010200000368058681 using {'batch_size': 500, 'epochs': 20}
0.008000 (0.000400) with: {'batch_size': 500, 'epochs': 10}
0.010200 (0.002200) with: {'batch_size': 500, 'epochs': 20}
[CV] END ..........................batch_size=500, epochs=10; total time=  16.5s

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

[CV] END ..........................batch_size=500, epochs=10; total time=  16.6s

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

[CV] END ..........................batch_size=500, epochs=20; total time=  17.0s

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

[CV] END ..........................batch_size=500, epochs=20; total time=  17.1s

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



## Data manipulation

For this model, the data will be separated into two layers; one for all the white pieces and one for the black pieces. The target will remain unchanged and there will still be no distinction between the pieces.

To get the data in the correct format, we can construct a $8\times8\times2$ numpy array to feed our model. The pieces which are moving will be provided in the 0th index of axis 2 (the array is structured [axis0, axis1, axis2]) and those not moving will be at index 1.

In [21]:
conv_move_list = [list({i for i in board.moves.all_valid}) for board in board_list]

conv_position_list = [
    (
        dict(board.loc_map), 
        board.to_move
    ) 
    for board in board_list
]

In [22]:
# Encode the location of any piece as a 1, and all other positions as 0
conv_feature = []
for instance in conv_position_list:
    # Set up empty array
    array = np.zeros((8, 8, 2))
    
    # Assign white and black to their correct layers
    if instance[1] == WHITE:
        # White to move
        WHITE_LAYER = 1
        BLACK_LAYER = 0
    else:
        # Black to move
        BLACK_LAYER = 1
        WHITE_LAYER = 0
    
    # Mark the correct locations in the matrix
    for location, piece in instance[0].items():
        if piece.colour == WHITE:
            array[location.i, location.j, WHITE_LAYER] = 1
        else:
            array[location.i, location.j, BLACK_LAYER] = 1

    conv_feature.append(array)

In [23]:
# We encode the valid "destination" squares with a 1 and invalid with a 0
conv_target = []
for instance in conv_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    conv_target.append(array.flatten())

In [24]:
X = np.array(conv_feature)
Y = np.array(conv_target)

# Split the data using sklearn train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

### Training a model

Now the data is in a format that contains much more information in a way that convolutional netoworks can process, we can define a convolutional model architecture (again using the Keras sequential API) to verify if this model can perform any better.

In [25]:
conv_model_1 = TFModel("conv_design_1")

conv_1_input = keras.Input(shape=(8, 8, 2))
cnv_1 = Conv2D(8, (3, 3), activation='relu')(conv_1_input)
cnv_1 = Conv2D(8, (3, 3), activation='relu')(cnv_1)
cnv_1 = Conv2D(8, (3, 3), activation='relu')(cnv_1)
cnv_1 = Flatten()(cnv_1)
cnv_1 = Dense(256, activation='relu')(cnv_1)
cnv_1 = Dense(128, activation='relu')(cnv_1)
conv_1_outputs = Dense(64, activation='sigmoid')(cnv_1)

conv_model_1.model = keras.Model(inputs=conv_1_input, outputs=conv_1_outputs)
conv_model_1.compile()
conv_model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 8, 8, 2)]         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 6, 6, 8)           152       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 8)           584       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 8)           584       
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               8448      
_________________________________________________________________
dense_4 (Dense)              (None, 128)               3289

2021-12-13 20:51:06.607484: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 20:51:06.607497: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 20:51:06.608568: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [26]:
conv_model_1.fit(X_train, Y_train)

Epoch 1/200
 1/80 [..............................] - ETA: 1:41 - loss: 0.6946 - accuracy: 0.0000e+00 - binary_accuracy: 0.4984

2021-12-13 20:51:07.821442: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-13 20:51:07.975885: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 20:51:07.975895: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 20:51:08.012965: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-13 20:51:08.017144: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-13 20:51:08.021128: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tensorboard_logs/conv_design_1_20211213-205106/train/plugins/profile/2021_12_13_20_51_08


20/80 [======>.......................] - ETA: 0s - loss: 0.6785 - accuracy: 0.0123 - binary_accuracy: 0.5925

2021-12-13 20:51:08.022473: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tensorboard_logs/conv_design_1_20211213-205106/train/plugins/profile/2021_12_13_20_51_08/Elliss-MacBook-Air.local.trace.json.gz
2021-12-13 20:51:08.025590: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tensorboard_logs/conv_design_1_20211213-205106/train/plugins/profile/2021_12_13_20_51_08
2021-12-13 20:51:08.025736: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for memory_profile.json.gz to tensorboard_logs/conv_design_1_20211213-205106/train/plugins/profile/2021_12_13_20_51_08/Elliss-MacBook-Air.local.memory_profile.json.gz
2021-12-13 20:51:08.026144: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: tensorboard_logs/conv_design_1_20211213-205106/train/plugins/profile/2021_12_13_20_51_08Dumped tool data for xplane.pb to tensorboard_logs/conv_design_1_2

80/80 [==============================] - 2s 10ms/step - loss: 0.6390 - accuracy: 0.0063 - binary_accuracy: 0.6521
Epoch 2/200
80/80 [==============================] - 1s 9ms/step - loss: 0.5517 - accuracy: 0.0028 - binary_accuracy: 0.7238
Epoch 3/200
80/80 [==============================] - 1s 9ms/step - loss: 0.5293 - accuracy: 0.0042 - binary_accuracy: 0.7344
Epoch 4/200
80/80 [==============================] - 1s 10ms/step - loss: 0.5196 - accuracy: 0.0074 - binary_accuracy: 0.7415
Epoch 5/200
80/80 [==============================] - 1s 9ms/step - loss: 0.5112 - accuracy: 0.0067 - binary_accuracy: 0.7491
Epoch 6/200
80/80 [==============================] - 1s 9ms/step - loss: 0.5031 - accuracy: 0.0090 - binary_accuracy: 0.7541
Epoch 7/200
80/80 [==============================] - 1s 9ms/step - loss: 0.4971 - accuracy: 0.0070 - binary_accuracy: 0.7591
Epoch 8/200
80/80 [==============================] - 1s 9ms/step - loss: 0.4902 - accuracy: 0.0093 - binary_accuracy: 0.7624
Epoch 9/20

80/80 [==============================] - 1s 9ms/step - loss: 0.3467 - accuracy: 0.0314 - binary_accuracy: 0.8464
Epoch 67/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3453 - accuracy: 0.0339 - binary_accuracy: 0.8459
Epoch 68/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3446 - accuracy: 0.0337 - binary_accuracy: 0.8471
Epoch 69/200
80/80 [==============================] - 1s 10ms/step - loss: 0.3402 - accuracy: 0.0342 - binary_accuracy: 0.8484
Epoch 70/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3414 - accuracy: 0.0365 - binary_accuracy: 0.8488
Epoch 71/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3408 - accuracy: 0.0346 - binary_accuracy: 0.8487
Epoch 72/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3356 - accuracy: 0.0303 - binary_accuracy: 0.8506
Epoch 73/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3368 - accuracy: 0.0398 - binary_accuracy: 0.8507
Epoc

80/80 [==============================] - 1s 9ms/step - loss: 0.2845 - accuracy: 0.0464 - binary_accuracy: 0.8768
Epoch 131/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2855 - accuracy: 0.0414 - binary_accuracy: 0.8760
Epoch 132/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2859 - accuracy: 0.0458 - binary_accuracy: 0.8758
Epoch 133/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2830 - accuracy: 0.0450 - binary_accuracy: 0.8776
Epoch 134/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2818 - accuracy: 0.0444 - binary_accuracy: 0.8784
Epoch 135/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2821 - accuracy: 0.0516 - binary_accuracy: 0.8780
Epoch 136/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2797 - accuracy: 0.0469 - binary_accuracy: 0.8791
Epoch 137/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2835 - accuracy: 0.0470 - binary_accuracy: 0.87

80/80 [==============================] - 1s 9ms/step - loss: 0.2601 - accuracy: 0.0541 - binary_accuracy: 0.8878
Epoch 195/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2533 - accuracy: 0.0514 - binary_accuracy: 0.8922
Epoch 196/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2540 - accuracy: 0.0553 - binary_accuracy: 0.8912
Epoch 197/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2525 - accuracy: 0.0542 - binary_accuracy: 0.8913
Epoch 198/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2532 - accuracy: 0.0521 - binary_accuracy: 0.8914
Epoch 199/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2501 - accuracy: 0.0607 - binary_accuracy: 0.8931
Epoch 200/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2515 - accuracy: 0.0589 - binary_accuracy: 0.8921


In [27]:
conv_model_1.evaluate(X_test, Y_test)

18/32 [===============>..............] - ETA: 0s - loss: 0.5712 - accuracy: 0.0243 - binary_accuracy: 0.7866

2021-12-13 20:53:33.358536: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 0s 7ms/step - loss: 0.5733 - accuracy: 0.0260 - binary_accuracy: 0.7845


The binary accuracy of this model does outperform the previous dense network used before, which is a promising result for this technique.

In [29]:
interact(
    generate_plots,
    model = fixed(conv_model_1),
    x = fixed(X_test),
    y = fixed(Y_test),
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)

interactive(children=(IntSlider(value=0, description='n', max=199, step=5), Checkbox(value=False, description=…

<function __main__.generate_plots(n, model=None, x=None, y=None, target=True, prediction=True, raw_prediction=True, diff=True)>

From these modified visualisation functions it is clear that there is greater performance by this model. The added implicit information on which pieces are moving, combined with the use of convolutional filters to properly extract features from the training data. Seeing a jump in binary accuracy as well as in many cases much more confident predictions from the model is indicitive that this approach is superior, due to the convolutional feature extraction layers

The behaviour of long range pieces is however still an issue for these systems.

# Convolutional network with more layers

It is now not unreasonable to think that the networks performance would be increased significantly if it were to be aware of which pieces are at which location; this is after all the most important deciding factor as to how they may move.

An approach to integrating this data is the following architecture;
- Input data (8 x 8 x 6), where there is one index on axis 2 for each of the five piece types (King, Queen, Rook, Knight, Bishop, Pawn) for the moving pieces, and another for the following
- The final layer of the input tensor will contain the position of the pieces which are not to move. Since the information on which type of piece these are is generally less important to predicting the moves, for now this can be left out.
- Additionally, passing the fully connected layer some information about whether the position is check for example may increase accuracy some more, since it will be able to tell when certain moves which usually are valid would not be valid. This information needs to reach the fully connected network but is not part of the convolutional network, so for this section the Keras functional API will be more appropriate as it offers more control.





## Data processing

In [39]:
# Encode the location of any piece as a 1, and all other positions as 0
conv_feature = []
piece_index_map = {
    'K': 0,
    'Q': 1,
    'R': 2,
    'B': 3,
    'N': 4,
    'P': 5
}
for instance in conv_position_list:
    # Set up empty array
    array = np.zeros((8, 8, 7))
    
    # Assign white and black to their correct layers
    if instance[1] == WHITE:
        # White to move
        MOVING_COLOUR = 1
    else:
        # Black to move
        MOVING_COLOUR = 0
    
    # Mark the correct locations in the matrix
    for location, piece in instance[0].items():
        if piece.colour == MOVING_COLOUR:
            array[location.i, location.j, piece_index_map[piece.kind]] = 1
        else:
            array[location.i, location.j, 6] = 1

    conv_feature.append(array)

In [40]:
# We encode the valid "destination" squares with a 1 and invalid with a 0
conv_target = []
for instance in conv_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    conv_target.append(array.flatten())

In [41]:
X = np.array(conv_feature)
Y = np.array(conv_target)

# Split the data using sklearn train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

## Create a model

In [42]:
conv_model_2 = TFModel("conv_design_2")

conv_2_input = keras.Input(shape=(8, 8, 7))
cnv_2 = Conv2D(10, (3, 3), activation='relu')(conv_2_input)
cnv_2 = Conv2D(10, (3, 3), activation='relu')(cnv_2)
cnv_2 = Conv2D(10, (3, 3), activation='relu')(cnv_2)
cnv_2 = Flatten()(cnv_2)
cnv_2 = Dense(256, activation='relu')(cnv_2)
cnv_2 = Dense(128, activation='relu')(cnv_2)
conv_2_outputs = Dense(64, activation='sigmoid')(cnv_2)

conv_model_2.model = keras.Model(inputs=conv_2_input, outputs=conv_2_outputs)
conv_model_2.compile()
conv_model_2.summary()


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 8, 8, 7)]         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 10)          640       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 4, 10)          910       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 2, 2, 10)          910       
_________________________________________________________________
flatten_2 (Flatten)          (None, 40)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               10496     
_________________________________________________________________
dense_10 (Dense)             (None, 128)               3289

2021-12-13 20:54:53.669462: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 20:54:53.669472: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 20:54:53.669687: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [43]:
conv_model_2.fit(X_train, Y_train)

Epoch 1/200
 3/80 [>.............................] - ETA: 2s - loss: 0.6925 - accuracy: 0.0022 - binary_accuracy: 0.5244     

2021-12-13 20:54:54.663361: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-13 20:54:54.806954: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 20:54:54.806963: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 20:54:54.847544: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-13 20:54:54.848015: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-13 20:54:54.848832: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tensorboard_logs/conv_design_2_20211213-205453/train/plugins/profile/2021_12_13_20_54_54
2021-12-13 20:54:54.849347: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tensorboard_logs/conv_design_2_20211213-205453/train/plugins/p

80/80 [==============================] - 2s 11ms/step - loss: 0.6416 - accuracy: 0.0097 - binary_accuracy: 0.6554
Epoch 2/200
80/80 [==============================] - 1s 11ms/step - loss: 0.5566 - accuracy: 0.0040 - binary_accuracy: 0.7161
Epoch 3/200
80/80 [==============================] - 1s 10ms/step - loss: 0.5260 - accuracy: 0.0022 - binary_accuracy: 0.7346
Epoch 4/200
80/80 [==============================] - 1s 10ms/step - loss: 0.5161 - accuracy: 0.0049 - binary_accuracy: 0.7419
Epoch 5/200
80/80 [==============================] - 1s 10ms/step - loss: 0.5057 - accuracy: 0.0079 - binary_accuracy: 0.7492
Epoch 6/200
80/80 [==============================] - 1s 10ms/step - loss: 0.4958 - accuracy: 0.0107 - binary_accuracy: 0.7547
Epoch 7/200
80/80 [==============================] - 1s 10ms/step - loss: 0.4894 - accuracy: 0.0153 - binary_accuracy: 0.7593
Epoch 8/200
80/80 [==============================] - 1s 10ms/step - loss: 0.4769 - accuracy: 0.0172 - binary_accuracy: 0.7675
Epoc

80/80 [==============================] - 1s 10ms/step - loss: 0.3053 - accuracy: 0.0318 - binary_accuracy: 0.8680
Epoch 66/200
80/80 [==============================] - 1s 10ms/step - loss: 0.3010 - accuracy: 0.0308 - binary_accuracy: 0.8701
Epoch 67/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2992 - accuracy: 0.0346 - binary_accuracy: 0.8718
Epoch 68/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2987 - accuracy: 0.0325 - binary_accuracy: 0.8704
Epoch 69/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2986 - accuracy: 0.0349 - binary_accuracy: 0.8716
Epoch 70/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2983 - accuracy: 0.0325 - binary_accuracy: 0.8715
Epoch 71/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2939 - accuracy: 0.0337 - binary_accuracy: 0.8738
Epoch 72/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2944 - accuracy: 0.0309 - binary_accuracy: 0.87

80/80 [==============================] - 1s 12ms/step - loss: 0.2311 - accuracy: 0.0451 - binary_accuracy: 0.9037
Epoch 130/200
80/80 [==============================] - 1s 13ms/step - loss: 0.2310 - accuracy: 0.0449 - binary_accuracy: 0.9040
Epoch 131/200
80/80 [==============================] - 1s 15ms/step - loss: 0.2305 - accuracy: 0.0374 - binary_accuracy: 0.9035
Epoch 132/200
80/80 [==============================] - 1s 13ms/step - loss: 0.2303 - accuracy: 0.0388 - binary_accuracy: 0.9038
Epoch 133/200
80/80 [==============================] - 1s 13ms/step - loss: 0.2325 - accuracy: 0.0384 - binary_accuracy: 0.9028
Epoch 134/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2261 - accuracy: 0.0337 - binary_accuracy: 0.9056
Epoch 135/200
80/80 [==============================] - 1s 13ms/step - loss: 0.2292 - accuracy: 0.0396 - binary_accuracy: 0.9044
Epoch 136/200
80/80 [==============================] - 1s 11ms/step - loss: 0.2280 - accuracy: 0.0408 - binary_accurac

80/80 [==============================] - 1s 9ms/step - loss: 0.1940 - accuracy: 0.0342 - binary_accuracy: 0.9210
Epoch 194/200
80/80 [==============================] - 1s 10ms/step - loss: 0.1948 - accuracy: 0.0428 - binary_accuracy: 0.9197
Epoch 195/200
80/80 [==============================] - 1s 10ms/step - loss: 0.1916 - accuracy: 0.0423 - binary_accuracy: 0.9214
Epoch 196/200
80/80 [==============================] - 1s 9ms/step - loss: 0.1982 - accuracy: 0.0399 - binary_accuracy: 0.9182
Epoch 197/200
80/80 [==============================] - 1s 9ms/step - loss: 0.1948 - accuracy: 0.0401 - binary_accuracy: 0.9207
Epoch 198/200
80/80 [==============================] - 1s 9ms/step - loss: 0.1932 - accuracy: 0.0395 - binary_accuracy: 0.9206
Epoch 199/200
80/80 [==============================] - 1s 10ms/step - loss: 0.1966 - accuracy: 0.0413 - binary_accuracy: 0.9188
Epoch 200/200
80/80 [==============================] - 1s 10ms/step - loss: 0.1924 - accuracy: 0.0450 - binary_accuracy: 0

In [44]:
conv_model_2.evaluate(X_test, Y_test)

18/32 [===============>..............] - ETA: 0s - loss: 0.5436 - accuracy: 0.0347 - binary_accuracy: 0.8192

2021-12-13 20:57:54.791643: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 0s 7ms/step - loss: 0.5377 - accuracy: 0.0320 - binary_accuracy: 0.8204


## Visualise the output

In [46]:
# From https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
interact(
    generate_plots,
    model = fixed(conv_model_2),
    x = fixed(X_test),
    y = fixed(Y_test),
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)

interactive(children=(IntSlider(value=0, description='n', max=199, step=5), Checkbox(value=False, description=…

<function __main__.generate_plots(n, model=None, x=None, y=None, target=True, prediction=True, raw_prediction=True, diff=True)>

Although this model performs approximately on par with the original fully connected network, we can see dramatic improvments in the ability to handle the motion of "long range" pieces - the model now has the ability to learn about which pieces can move in which axis. 